In [8]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import sidetable as stb
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
import nltk
from sklearn.decomposition import NMF
import random
random.seed(123)
np.random.seed(123)

In [9]:
df1 = pd.read_csv('../data/validation.csv',skipinitialspace=True)
df1 = df1.rename(columns = lambda x: x.strip())
df1.head()

,avg_rating,book_title,author_num_unique_books,book_description,book_reviews,genres_str,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,reviews_per_month,reviews_per_month_log
0,4.22,"Good Vibes, Good Life",5.0,A beautifully designed book full of inspiring ...,Vex Kings masterpiece is incredible! The book ...,audiobook health mental-health non-fiction per...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.708333,2.815908
1,3.94,Blink,68.0,Drawing on cutting-edge neuroscience and psych...,Blink is- what all the stories case studies ...,audiobook business leadership non-fiction pers...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,84.144444,4.432535
2,4.03,The Secret Letters of the Monk Who Sold His Fe...,83.0,Title: Secret Letters of the Monk Who Sold His...,I have always believed in people coming into y...,business contemporary inspirational leadership...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.268519,1.661749
3,3.98,The Memory Book,48.0,Unleash the hidden power of your mind through ...,I am giving this book a very high rating becau...,brain education health how-to non-fiction pers...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.226449,-1.485234
4,3.41,52 Small Changes,10.0,"Whether as New Year’s resolutions, birthday wi...",So Im going to cheat a bit and review this boo...,health inspirational mental-health non-fiction...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.074074,0.071459


In [10]:
# author will input title and description
df2 = df1.copy()
df2['title_description'] = df1.apply(lambda x: preprocess(x['book_title'] + ' '+x['book_description']),axis=1)
df2['reviews_genres'] = df1.apply(lambda x: preprocess(x['book_reviews'] + ' '+x['genres_str']),axis=1)

In [11]:
df2.head()

,avg_rating,book_title,author_num_unique_books,book_description,book_reviews,genres_str,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,reviews_per_month,reviews_per_month_log,title_description,reviews_genres
0,4.22,"Good Vibes, Good Life",5.0,A beautifully designed book full of inspiring ...,Vex Kings masterpiece is incredible! The book ...,audiobook health mental-health non-fiction per...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16.708333,2.815908,good vibe good life beauti design full inspi...,vex king masterpiec incr equal part person g...
1,3.94,Blink,68.0,Drawing on cutting-edge neuroscience and psych...,Blink is- what all the stories case studies ...,audiobook business leadership non-fiction pers...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,84.144444,4.432535,blink draw cut edg neurosci psycholog display...,blink stori case studi argument add attempt u...
2,4.03,The Secret Letters of the Monk Who Sold His Fe...,83.0,Title: Secret Letters of the Monk Who Sold His...,I have always believed in people coming into y...,business contemporary inspirational leadership...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.268519,1.661749,secret letter monk sold ferrari titl secret l...,alway believ person come life reason get han...
3,3.98,The Memory Book,48.0,Unleash the hidden power of your mind through ...,I am giving this book a very high rating becau...,brain education health how-to non-fiction pers...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.226449,-1.485234,memori unleash hidden power mind harri loray...,give high rate think read cant rememb im gue...
4,3.41,52 Small Changes,10.0,"Whether as New Year’s resolutions, birthday wi...",So Im going to cheat a bit and review this boo...,health inspirational mental-health non-fiction...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.074074,0.071459,fifti two small chang whether new resolut bi...,im go cheat bit review ive complet finish ca...


In [12]:
# topics are not accurate, recalculate them
# load topic model
import pickle
cutoff_topic = 0.1
num_topics = 10
filename_model = '../data/topic_model_tfidf_nmf.pickle'
nmf_model = pickle.load(open(filename_model, 'rb'))
filename_model = '../data/topic_model_tfidf.pickle'
tfidf_model = pickle.load(open(filename_model, 'rb'))
# compute new topic vectors
new_topics = []
for stri in df2['title_description'].values:
    new_topic_i = nmf_model.transform(tfidf_model.transform([stri]))[0]
    for i in range(0,len(new_topic_i)):
        new_topic_i[i] = 0 if new_topic_i[i] < cutoff_topic else 1
    new_topics.append(new_topic_i)
# pd.DataFrame(new_topics)
df_x_nmf = pd.DataFrame(new_topics,columns = ['topic '+str(i) for i in range(0,num_topics)]) 
df_x_nmf.head()

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
### insert new topics into df2
df2_new_topic = df2.copy()
for col_i in ['topic '+str(i) for i in range(0,num_topics)]:
    df2_new_topic[col_i] = df_x_nmf.apply(lambda x: x[col_i], axis = 1)
df2_new_topic.head()

,avg_rating,book_title,author_num_unique_books,book_description,book_reviews,genres_str,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,reviews_per_month,reviews_per_month_log,title_description,reviews_genres
0,4.22,"Good Vibes, Good Life",5.0,A beautifully designed book full of inspiring ...,Vex Kings masterpiece is incredible! The book ...,audiobook health mental-health non-fiction per...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,16.708333,2.815908,good vibe good life beauti design full inspi...,vex king masterpiec incr equal part person g...
1,3.94,Blink,68.0,Drawing on cutting-edge neuroscience and psych...,Blink is- what all the stories case studies ...,audiobook business leadership non-fiction pers...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,84.144444,4.432535,blink draw cut edg neurosci psycholog display...,blink stori case studi argument add attempt u...
2,4.03,The Secret Letters of the Monk Who Sold His Fe...,83.0,Title: Secret Letters of the Monk Who Sold His...,I have always believed in people coming into y...,business contemporary inspirational leadership...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.268519,1.661749,secret letter monk sold ferrari titl secret l...,alway believ person come life reason get han...
3,3.98,The Memory Book,48.0,Unleash the hidden power of your mind through ...,I am giving this book a very high rating becau...,brain education health how-to non-fiction pers...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226449,-1.485234,memori unleash hidden power mind harri loray...,give high rate think read cant rememb im gue...
4,3.41,52 Small Changes,10.0,"Whether as New Year’s resolutions, birthday wi...",So Im going to cheat a bit and review this boo...,health inspirational mental-health non-fiction...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.074074,0.071459,fifti two small chang whether new resolut bi...,im go cheat bit review ive complet finish ca...


In [14]:
## filter books with more than one topic, less than cut_num_reviews reviews per month, and author published less than cut_num_books_author books
cut_num_reviews = 5
cut_num_books_author = 10

df3 = df2_new_topic.copy()
df3['sum_topics'] = df2_new_topic.apply(lambda x: x['topic 0']+x['topic 1']+x['topic 2']+x['topic 3']+x['topic 4']+x['topic 5']+x['topic 6']+x['topic 7']+x['topic 8']+x['topic 9'],axis=1)
# df4 = df3[(df3['topic 0'] < 1)&(df3['topic 2'] < 1)&(df3['topic 6'] < 1)&(df3['sum_topics'] > 0.0) & (df3['reviews_per_month']<cut_num_reviews) & (df3['author_num_unique_books']< cut_num_books_author)]
df4 = df3.copy() # TODO
df4.head()

,avg_rating,book_title,author_num_unique_books,book_description,book_reviews,genres_str,topic 0,topic 1,topic 2,topic 3,...,topic 5,topic 6,topic 7,topic 8,topic 9,reviews_per_month,reviews_per_month_log,title_description,reviews_genres,sum_topics
0,4.22,"Good Vibes, Good Life",5.0,A beautifully designed book full of inspiring ...,Vex Kings masterpiece is incredible! The book ...,audiobook health mental-health non-fiction per...,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,16.708333,2.815908,good vibe good life beauti design full inspi...,vex king masterpiec incr equal part person g...,2.0
1,3.94,Blink,68.0,Drawing on cutting-edge neuroscience and psych...,Blink is- what all the stories case studies ...,audiobook business leadership non-fiction pers...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.144444,4.432535,blink draw cut edg neurosci psycholog display...,blink stori case studi argument add attempt u...,1.0
2,4.03,The Secret Letters of the Monk Who Sold His Fe...,83.0,Title: Secret Letters of the Monk Who Sold His...,I have always believed in people coming into y...,business contemporary inspirational leadership...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.268519,1.661749,secret letter monk sold ferrari titl secret l...,alway believ person come life reason get han...,0.0
3,3.98,The Memory Book,48.0,Unleash the hidden power of your mind through ...,I am giving this book a very high rating becau...,brain education health how-to non-fiction pers...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.226449,-1.485234,memori unleash hidden power mind harri loray...,give high rate think read cant rememb im gue...,0.0
4,3.41,52 Small Changes,10.0,"Whether as New Year’s resolutions, birthday wi...",So Im going to cheat a bit and review this boo...,health inspirational mental-health non-fiction...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.074074,0.071459,fifti two small chang whether new resolut bi...,im go cheat bit review ive complet finish ca...,0.0


In [15]:
### create column with topics of book by index
random.seed(123)
np.random.seed(123)
def get_topics(row_i):
    topic_nums = ""
    for i in range(0,10):
        if row_i['topic '+str(i)] > 0:
            topic_nums += str(i)+' '
    return topic_nums

df5 = df4.copy()
df5['topic_indices'] = df5.apply(lambda x: get_topics(x),axis = 1)
df5.head(n=30).sample(frac=1)[['book_title','book_description','topic_indices']]

,book_title,book_description,topic_indices
7,Your Best Year Ever,We all want to live a life that matters. We al...,7
29,Burnout,This groundbreaking book explains why women ex...,2
5,Peaks and Valleys,وجود دره و قله در زندگی برای همه طبیعی است.حال...,
26,Rising Strong,The physics of vulnerability is simple: If we ...,3
8,You Are Not Your Brain,Two neuroscience experts explain how their 4-S...,4
27,The Power of Full Engagement,“Combines the gritty toughmindedness of the be...,6
12,Trauma and Recovery,When Trauma and Recovery was first published i...,
21,The Desire Map,The Desire Map program is just that — a progra...,7
11,Resisting Happiness,"A true story about why we sabotage ourselves, ...",5
23,The Rules of Life,"Think about the world’s happiest, most content...",


In [16]:
# tokenize and stem topic words
topic_names = ['human problem question study present effect reason fact ask inform listen real happen', # 0, not significant
               '* relationship family problem', # 1
               'women', # 2, not significant
               '* story inspire family happen real listen', # 3
               '* brain study inform effect fact', # 4
               '* happy positive inspire present happen reason', # 5 
               'emotion step present positive effect', # 6 not significant
               '* success goal business create positive inspire', # 
               '* habit step goal effect problem inform',
               '* advice job business listen step inform family']
df_temp = pd.DataFrame(topic_names)
topic_names = [preprocess(stri) for stri in topic_names]

In [17]:
# create sets
df6 = df5.copy()
def get_topics_str(row_i):
    topic_names_all = ""
    for i in row_i['topic_indices'].split():
        topic_names_all += topic_names[int(i)]+' '
    return set(topic_names_all.split())

# df6['topics_str'] = df5.apply(lambda x: " ".join(set((topic_names[int(x['topic_indices'].split()[0])]+ ' ' +topic_names[int(x['topic_indices'].split()[1])]).split())), axis = 1)
df6['topics_str'] = df5.apply(lambda x: " ".join(get_topics_str(x)), axis = 1)

# df6['topic_0_str'] = df5.apply(lambda x: topic_names[int(x['topic_indices'].split()[0])],axis = 1)
# df6['topic_1_str'] = df5.apply(lambda x: topic_names[int(x['topic_indices'].split()[1])],axis = 1)

df6['title_description'] = df5.apply(lambda x: " ".join(set(x['title_description'].split())),axis = 1)
df6['reviews_genres'] = df5.apply(lambda x: " ".join(set(x['reviews_genres'].split())),axis = 1)
df6.head()
# df6.topics_str.values

,avg_rating,book_title,author_num_unique_books,book_description,book_reviews,genres_str,topic 0,topic 1,topic 2,topic 3,...,topic 7,topic 8,topic 9,reviews_per_month,reviews_per_month_log,title_description,reviews_genres,sum_topics,topic_indices,topics_str
0,4.22,"Good Vibes, Good Life",5.0,A beautifully designed book full of inspiring ...,Vex Kings masterpiece is incredible! The book ...,audiobook health mental-health non-fiction per...,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,16.708333,2.815908,univ world posit young happi good overcom oppo...,world failur think delight wont law long short...,2.0,5 8,posit reason happi step problem happen habit i...
1,3.94,Blink,68.0,Drawing on cutting-edge neuroscience and psych...,Blink is- what all the stories case studies ...,audiobook business leadership non-fiction pers...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,84.144444,4.432535,world think tip fault failur edg hard coach pe...,thinker بزرگ، seen می‌زنند؟ kid دچار دافعه pre...,1.0,4,studi brain inform effect fact
2,4.03,The Secret Letters of the Monk Who Sold His Fe...,83.0,Title: Secret Letters of the Monk Who Sold His...,I have always believed in people coming into y...,business contemporary inspirational leadership...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.268519,1.661749,secret monk letter titl sold ferrari element b...,world hang fair primari person suddenli alway ...,0.0,,
3,3.98,The Memory Book,48.0,Unleash the hidden power of your mind through ...,I am giving this book a very high rating becau...,brain education health how-to non-fiction pers...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.226449,-1.485234,phone number school right shorten power situat...,world amount hang fair seen tomb stone reach r...,0.0,,
4,3.41,52 Small Changes,10.0,"Whether as New Year’s resolutions, birthday wi...",So Im going to cheat a bit and review this boo...,health inspirational mental-health non-fiction...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.074074,0.071459,posit exerci quick resolut surprisingli wish s...,world amount seen resolut reach wish amazon ki...,0.0,,


In [18]:
df_temp

,0
0,human problem question study present effect re...
1,* relationship family problem
2,women
3,* story inspire family happen real listen
4,* brain study inform effect fact
5,* happy positive inspire present happen reason
6,emotion step present positive effect
7,* success goal business create positive inspire
8,* habit step goal effect problem inform
9,* advice job business listen step inform family


In [19]:
# are there words on the topics that are in reviews_genres and topics_str, but not in title_description
df7 = df6.copy()
df7['words_to_add'] = df6.apply(lambda x: " ".join(set(x['topics_str'].split()).intersection(set(x['reviews_genres'].split())) - set(x['title_description'].split())),axis = 1)
df7 = df7[df7['topic_indices']=='7 9 ']
df7[['book_title','book_description','words_to_add','topic_indices','author_num_unique_books','reviews_per_month']]

,book_title,book_description,words_to_add,topic_indices,author_num_unique_books,reviews_per_month
9,#Girlboss,In the New York Times bestseller that the Wash...,posit busi creat inspir,7 9,7.0,57.777778
37,The Renaissance Soul,How do I decide what to do with my life when t...,posit step busi inform goal creat,7 9,2.0,1.101190
178,The Greatness Guide,Each one of us is called to greatness. We can ...,posit famili step listen inform goal job creat,7 9,83.0,2.946429
227,The Antidote,"A witty, fascinating, and counterintuitive rea...",posit listen inform job creat inspir,7 9,10.0,11.364583


In [20]:
## testing on Barnes and Noble data, very different from validation
df_bn = pd.read_csv('../data/barnes_and_noble.csv',skipinitialspace=True)
df_bn = df_bn.rename(columns = lambda x: x.strip())
df_bn.head()

,book_title,book_description
0,5 Secrets of The Matrix,My many years of life coaching and family - re...
1,Alcoholics Anonymous 4th Edition,Originally published in 1939 when A.A. member...
2,Changed From The Inside Out Loving God,How loving and understanding God will transfor...
3,Emotional Eating Solution,Are You Frustrated With Uncontrollable Food Cr...
4,Finding Me,Sheena Sullivan Morelli and her sisters Darcy...


In [24]:
# preprocess
df_bn_2 = df_bn.copy()
df_bn_2['title_description'] = df_bn.apply(lambda x: preprocess(str(x['book_title']) + ' '+str(x['book_description'])),axis=1)
df_bn_2.head()

,book_title,book_description,title_description
0,5 Secrets of The Matrix,My many years of life coaching and family - re...,secret matrix mani life coach famili relatio...
1,Alcoholics Anonymous 4th Edition,Originally published in 1939 when A.A. member...,alcohol anonym 4th edit origin publish one n...
2,Changed From The Inside Out Loving God,How loving and understanding God will transfor...,chang insid love god love understand god tran...
3,Emotional Eating Solution,Are You Frustrated With Uncontrollable Food Cr...,emot eat solut frustrat uncontrol food crave ...
4,Finding Me,Sheena Sullivan Morelli and her sisters Darcy...,find sheena sullivan morelli sister darci reg...


In [29]:
# compute new topic vectors
new_topics_bn = []
for stri in df_bn_2['title_description'].values:
    new_topic_i = nmf_model.transform(tfidf_model.transform([stri]))[0]
    for i in range(0,len(new_topic_i)):
        new_topic_i[i] = 0 if new_topic_i[i] < cutoff_topic else 1
    new_topics_bn.append(new_topic_i)
# pd.DataFrame(new_topics)
df_x_nmf_bn = pd.DataFrame(new_topics_bn,columns = ['topic '+str(i) for i in range(0,num_topics)]) 
df_x_nmf_bn.head()

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
### insert new topics into df2
df_bn_3 = df_bn_2.copy()
for col_i in ['topic '+str(i) for i in range(0,num_topics)]:
    df_bn_3[col_i] = df_x_nmf_bn.apply(lambda x: x[col_i], axis = 1)
df_bn_3.head()

,book_title,book_description,title_description,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9
0,5 Secrets of The Matrix,My many years of life coaching and family - re...,secret matrix mani life coach famili relatio...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Alcoholics Anonymous 4th Edition,Originally published in 1939 when A.A. member...,alcohol anonym 4th edit origin publish one n...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Changed From The Inside Out Loving God,How loving and understanding God will transfor...,chang insid love god love understand god tran...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Emotional Eating Solution,Are You Frustrated With Uncontrollable Food Cr...,emot eat solut frustrat uncontrol food crave ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,Finding Me,Sheena Sullivan Morelli and her sisters Darcy...,find sheena sullivan morelli sister darci reg...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
### create column with topics of book by index
random.seed(123)
np.random.seed(123)
def get_topics(row_i):
    topic_nums = ""
    for i in range(0,10):
        if row_i['topic '+str(i)] > 0:
            topic_nums += str(i)+' '
    return topic_nums

df_bn_4 = df_bn_3.copy()
df_bn_4['topic_indices'] = df_bn_3.apply(lambda x: get_topics(x),axis = 1)
df_bn_4.head(n=30).sample(frac=1)[['book_title','book_description','topic_indices']]

,book_title,book_description,topic_indices
7,How to Study the Bible,I will never forget thy precepts; for with the...,
29,Your True Power,Do you want to live your most exceptional life...,2 3
5,Goals How to Set Goals You ll Actually Achiev...,Have you ever experienced frustration when fai...,7
26,Why Doesn t God Speak to Me?,Eager hearts cry out hoping for a single word...,
8,I Deserve a Donut (And Other Lies That Make Yo...,When you re in a tempting situation all you c...,
27,Winter s Secret,Some secrets chill to the bone....Smashed heir...,
12,My People the Amish The True Story of an Ami...,In My People the Amish Joe Keim paints a det...,3
21,The Way to God,If we confess our sins he is faithful and jus...,
11,Master Your Mindpower,Stéphane and Shalee Schafeitel Co-founders of...,6 7
23,Understanding and Healing Trauma (The Fluff-Fr...,Did you have a difficult childhood? If so you...,


In [21]:
import nltk
nltk.download('punkt')
def convert_lower_case(data):
    return np.char.lower(data)
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text
def remove_obvious(data):
    remove_words = ['audio', 'page','book','audiobook', 'self', 'help','author','bestel','reader','year','thousand','twenti'] #,'want','work','way','world','use']
    return_data = str(data)
    for ri in remove_words:
        return_data = return_data.replace(ri,"")
    return_data = return_data.replace("peopl","person")
    return return_data 

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    data = remove_obvious(data)
    return data

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\valer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
